# Simple Linear versus Ridge Regression 

## Step 1:  Getting, understanding, and preprocessing the dataset

We first import the standard libaries and some libraries that will help us scale the data and perform some "feature engineering" by transforming the data into $\Phi_2({\bf x})$

In [27]:
import numpy as np
import sklearn
from sklearn.datasets import load_boston
from sklearn.preprocessing import PolynomialFeatures
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import sklearn.linear_model
from sklearn.model_selection import KFold
import copy

###  Importing the dataset

In [28]:
# Import the boston dataset from sklearn
# Load dataset to some variable 

boston_data = load_boston()

In [40]:
# Create X and Y variables - X holding the .data and Y holding .target

x = boston_data.data
y = boston_data.target

# Reshape Y to be a rank 2 matrix using y.reshape()
#y=y.reshape(-1,2)
#print("Rank of Y:", np.linalg.matrix_rank(y))
#y=y.reshape(-1,1) #to maintain (506, 1) shape
#print(y.shape)

# Observe the number of features and the number of labels
num_features = x.shape[1]
num_examples = x.shape[0]
feature_names = boston_data.feature_names
first_two_rows = x[0:2]

print("The number of fearutes is: ", num_features)
# Printing out the features
print("The features: ", feature_names)
# The number of examples
print("Number of examples in our dataset: ", num_examples)
# Observing the first 2 rows of the data
print("First two rows: ", first_two_rows)


Rank of Y: 2
The number of fearutes is:  13
The features:  ['CRIM' 'ZN' 'INDUS' 'CHAS' 'NOX' 'RM' 'AGE' 'DIS' 'RAD' 'TAX' 'PTRATIO'
 'B' 'LSTAT']
Number of examples in our dataset:  506
First two rows:  [[6.3200e-03 1.8000e+01 2.3100e+00 0.0000e+00 5.3800e-01 6.5750e+00
  6.5200e+01 4.0900e+00 1.0000e+00 2.9600e+02 1.5300e+01 3.9690e+02
  4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 0.0000e+00 4.6900e-01 6.4210e+00
  7.8900e+01 4.9671e+00 2.0000e+00 2.4200e+02 1.7800e+01 3.9690e+02
  9.1400e+00]]


We will also create polynomial feeatures for the dataset to test linear and ridge regression on data with d = 1 and data with d = 2. Feel free to increase the # of degress and see what effect it has on the training and test error. 

In [33]:
# Create a PolynomialFeatures object with degree = 2. Using PolynomialFeatures(degree=2)
# Transform X and save it into X_2 using poly.fit_transform(X)
# Simply copy Y into Y_2 

# I created a function so I am able to call it when predicting a new case

def transform_poly(x):   
    poly = PolynomialFeatures(degree=2)
    x2=poly.fit_transform(x)
    y2=copy.deepcopy(y)
    
    return x2


In [31]:
# the shape of X_2 and Y_2 - should be (506, 105) and (506, 1) respectively
# because I created a function, this is not able to execute
# the result is when x2 and y2 are not a part of a function

print(x2.shape)
print(y2.shape)

(506, 105)
(506, 1)


# Your code goes here

In [34]:
# Define the get_coeff_ridge_normaleq function. Use the normal equation method.
# Return w values
# I created one function: for getting the linear coeff, we can pass alpha as 0
# and we don't need to check whether it is linear or ridge in the k_fold validation
# it is simpler to distingush them when accumulating the results

def get_coeff_linear_ridge_normaleq(x_train, y_train, alpha):
    if len(y_train.shape)==1:
        y_train = y_train[:,np.newaxis]
        
    m = alpha*np.identity(x_train.shape[1])
    n = np.linalg.pinv(np.matmul(np.transpose(x_train), x_train)+m)
    
    w=np.matmul(np.matmul(n,np.transpose(x_train)), y_train)
    
    return w

In [35]:
# Define the evaluate_err_ridge function.
# Return the train_error and test_error values

def evaluate_err(x_train, x_test, y_train, y_test, w): 
    pred_train = (np.matmul(x_train, w).squeeze())*np.std(y_train) + np.mean(y_train)
    pred_test = (np.matmul(x_test, w).squeeze())*np.std(y_train) + np.mean(y_train)
    
    temp_train = (y_train-pred_train)**2
    temp_test = (y_test-pred_test)**2
    
    size_train = y_train.shape[0]
    size_test = y_test.shape[0]
    
    train_error = np.sum(temp_train)/size_train
    test_error = np.sum(temp_test)/size_test
    
    return train_error, test_error

In [41]:
# Finish writting the k_fold_cross_validation function. 
# Returns the average training error and average test error from the k-fold cross validation
# Sklearns K-Folds cross-validator: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

def k_fold_cross_validation(k, x, y, alpha):
    
    kf = KFold(n_splits=k, random_state=21, shuffle=True)
    total_E_val_test = []
    total_E_val_train = []
    
    for train_index, test_index in kf.split(x):
        x_train, x_test = x[train_index], x[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # Centering the data so we do not need the intercept term (we could have also chose w_0=average y value)
        # Scaling the data matrix
        scaler = preprocessing.StandardScaler().fit(x_train)
        x_train = scaler.transform(x_train)
        x_test = scaler.transform(x_test)
        
        x_train_mean = np.mean(x_train)
        x_train_std = np.std(x_train)
        x_train_std = np.where(x_train_std == 0, 1, x_train_std)
        
        x_train = (x_train - x_train_mean)/x_train_std
        x_train_temp = np.ones((x_train.shape[0],x.shape[1] + 1))
        x_train_temp[:, :x.shape[1]] = x_train
        
        x_test = (x_test - x_train_mean)/x_train_std
        x_test_temp = np.ones((x_test.shape[0], x.shape[1] + 1))
        x_test_temp[:, :x.shape[1]]=x_test
        
        y_train_mean = np.mean(y_train)
        y_train_std = np.std(y_train)
        y_train_temp = (y_train - y_train_mean)/y_train_std
        
        
        
        # Determine the training error and the test error
        
        w = get_coeff_linear_ridge_normaleq(x_train_temp, y_train_temp, alpha)
        
        total_E_val_test_temp, total_E_val_train_temp = evaluate_err(x_train_temp, x_test_temp, y_train, y_test, w)
        
        total_E_val_test.append(total_E_val_test_temp)
        total_E_val_train.append(total_E_val_train_temp)
        
 
    return  total_E_val_test, total_E_val_train, w
    


In [37]:
# print the error for the both linear regression and ridge regression
# the error should include both training error and testing error

test_E_val_linear, train_E_val_linear, w = k_fold_cross_validation(10, x, y, 0)

#print(len(test_E_val_linear[0]))

print("Linear Testing Err ", np.sum(test_E_val_linear)/len(test_E_val_linear))
print("Linear Training Err ", np.sum(train_E_val_linear)/len(train_E_val_linear))

alpha_range = np.logspace(1, 7, num=13)
fin_alpha = 0
min_test_E = float("inf")
min_train_E = float("inf")

for i in alpha_range:
    test_E_val_ridge, train_E_val_ridge, w = k_fold_cross_validation(10, x, y, i)
    avg_test_E = np.sum(test_E_val_ridge)/10
    avg_train_E = np.sum(test_E_val_ridge)/10
    
    if min_test_E > avg_test_E:
        min_test_E = avg_test_E
        min_train_E = avg_train_E
        fin_alpha = i
        

print("Best alpha: ", fin_alpha)
print("Ridge Testing Err ", min_test_E)
print("Ridge Training Err ", min_train_E)
    

Linear Testing Err  21.806183575851065
Linear Training Err  23.63606860542815
Best alpha:  10.0
Ridge Testing Err  21.892901157570186
Ridge Training Err  21.892901157570186


In [38]:
# use the model to predict the new test case.

x_t = transform_poly(x)
x_t = x_t[:,1:]

trein_E_val_linear, Test_E_val_linear, w = k_fold_cross_validation(10, x_t, y, 0)

print("Linear Testing Err ", np.sum(test_E_val_linear)/len(test_E_val_linear))
print("Linear Training Err ", np.sum(train_E_val_linear)/len(train_E_val_linear))

alpha_range = np.logspace(1, 7, num=13)
fin_alpha = 0
min_test_E = float("inf")
min_train_E = float("inf")

for i in alpha_range:
    test_E_val_ridge, train_E_val_ridge, w = k_fold_cross_validation(10, x_t, y, i)
    avg_test_E = np.sum(test_E_val_ridge)/10
    avg_train_E = np.sum(test_E_val_ridge)/10
    
    if min_test_E > avg_test_E:
        min_test_E = avg_test_E
        min_train_E = avg_train_E
        best_alpha = i
        best_w = w
        

print("Best alpha: ", best_alpha)
print("Ridge Testing Err ", min_test_E)
print("Ridge Training Err ", min_train_E)


Linear Testing Err  21.806183575851065
Linear Training Err  23.63606860542815
Best alpha:  10.0
Ridge Testing Err  10.049055874118878
Ridge Training Err  10.049055874118878


In [39]:
# I would choose the closed form ridge regression model with a second degree
# polynomial transformation, as it gives the lowes testing error, with almost
# no difference with the training error.
# Here are the parameters:

print("K: ", 10)
print("Alpha: ", best_alpha)
print("w: ", best_w.squeeze())

K:  10
Alpha:  10.0
w:  [ 1.93291631e-02 -5.29607482e-02  8.44852208e-02  3.87773560e-02
  3.24937182e-02  1.93839150e-01  5.94868364e-02 -1.38933143e-01
  1.55683231e-01  4.91859206e-02 -6.98996045e-03  3.07602122e-02
  2.32052470e-03  6.92541346e-02  1.91388744e-02  2.50098096e-02
  3.40009806e-01 -5.69925239e-02 -8.79819469e-02  1.99377459e-02
 -8.06025507e-02 -1.30195952e-02  4.56698349e-03  1.16098288e-02
 -6.11411451e-02 -5.42253757e-03  3.23265195e-02 -5.81087633e-02
 -1.91430078e-02 -5.34896726e-03  7.13576247e-02  9.12165307e-04
 -1.73166181e-02 -2.08731570e-02  7.11431920e-02  7.26865184e-02
 -2.91750474e-02 -8.44708535e-02  6.49981833e-02 -3.88996151e-02
  3.19892899e-02 -8.73029389e-02  6.76643816e-02 -1.09684224e-01
  8.26411714e-02  9.88321892e-02 -2.46958249e-02  8.20147916e-02
 -1.57930949e-01  3.87773560e-02 -1.93594223e-01 -1.23308305e-01
  6.42956527e-02  1.24583395e-01 -1.53006206e-01 -9.28927521e-02
  2.91773001e-02  1.63274533e-01 -4.34923293e-02 -3.84192880e-02
 